In [2]:


import pandas as pd
import numpy as np



In [3]:


import warnings
warnings.filterwarnings('ignore')



In [4]:


from datetime import datetime
from datetime import date



In [5]:


#Get Data month (previous month), and 13 months before data month
today = date.today()
data_month = today.month - 1
data_month_year = (date.today() - pd.offsets.DateOffset(months=1)).year
start_month = (date.today() - pd.offsets.DateOffset(months=13)).month
start_year = (date.today() - pd.offsets.DateOffset(months=13)).year

try:
    df_ins = pd.read_excel(r"C:\Users\pragyan.agrawal\Downloads\RELIANCERX Ins.xlsx")
except:
    print("Enter correct file path for Combined Ins File")

df_combined_ins = df_ins.copy()



In [6]:


#Sender Name Filter
df_combined_ins = df_combined_ins[df_combined_ins['SENDER_NAME'].isin(['Amerisource','BIOCARE','Cardinal','CARDSD','Cesar Castillo','CIBD','CuraScript','DMS','Dakota Drug','FFF','HARVARD','Henry Schien','ICS','McKesson','OTN','Mckesson PB','Metro Medical','Morris and Dickson','MUTUALDRUG','Prescription Supply','Rochester Drug Cooperative','SMITH','TAP','Value Drug'])]

df_combined_ins["month"] = df_combined_ins.FILE_CONTENT_START_DATE.dt.month
df_combined_ins["year"] = df_combined_ins.FILE_CONTENT_START_DATE.dt.year



In [7]:


#converting date format into a standard format and storing it in a new column
df_combined_ins['DATE'] = pd.to_datetime(pd.to_datetime(df_combined_ins['FILE_CONTENT_START_DATE']).dt.strftime('%Y-%m-%d'))
# df_combined_ins.rename(columns = {'ID1_VALUE.1':'BRANCH_ID'}, inplace = True)



In [8]:


start_check_date = str(start_year) + '-' + str(start_month) + '-01'

if data_month in [1,3,5,7,8,10,12]:
    end_check_date = str(data_month_year) + '-' + str(data_month) + '-31'
elif data_month in [2]:
    end_check_date = str(data_month_year) + '-' + str(data_month) + '-28'
else:
    end_check_date = str(data_month_year) + '-' + str(data_month) + '-30'



In [9]:


# create pivot with branch_id as index:
ins_pivot_branch = pd.pivot_table(df_combined_ins.loc[(df_combined_ins['DATE'] >= start_check_date) & (df_combined_ins['DATE'] <= end_check_date)], values='QTY_DISPENSED', index='ID1_VALUE.1', columns=['year','month'], aggfunc=np.sum,fill_value=0)
ins_pivot_branch = ins_pivot_branch.reset_index()
ins_pivot_branch = ins_pivot_branch.set_index(ins_pivot_branch.columns[0])



In [10]:


ins_pivot_branch



year            2022           2023                                          \
month             10   11   12    1    2    3    4    5    6    7    8    9   
(ID1_VALUE.1, )                                                               
FS2807088        186  182  194  226  177  211  254  291  240  246  314  208   
G24KKWMF1        231  254  251  372  254  328  321  395  575  276  505  353   

year                  
month             10  
(ID1_VALUE.1, )       
FS2807088        192  
G24KKWMF1        510

In [11]:


# output to csv file:
ins_pivot_branch.to_csv('ins_pivot_branch.csv')



In [12]:


import pandas as pd
import numpy as np

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

df = None
orig_df_current_month_branch_dq = None
orig_df_previous_month_branch_dq = None
branch_report_file_path=r"C:\Users\pragyan.agrawal\Downloads\RELIANCERX BRANCH.xlsx"
def dq_non_trending_branch_analysis(branch_report_file_path,current_month_branch_dq_file_path,previous_month_branch_dq_file_path):
    try:
        df = pd.read_excel(branch_report_file_path)
    except:
        print("Please enter correct path for Branch Report file")
    try:
        orig_df_current_month_branch_dq = pd.read_excel(current_month_branch_dq_file_path)
    except:
        print("Please enter correct path for Current Month Branch DQ file")
    try:
        orig_df_previous_month_branch_dq = pd.read_excel(previous_month_branch_dq_file_path)
    except:
        print("Please enter correct path for Previous Month Branch DQ file")

    if df is not None:
        def transform_columns(column):
            if column == 'Unnamed: 0':
                return 'SENDER NAME'
            elif column == 'Unnamed: 1':
                return 'BRANCH ID'
            elif column == 'Unnamed: 2':
                return 'NDC'
            elif column == 'Unnamed: 3':
                return 'PRODUCT INFO'
            else:
                return column

        # Apply the column transformation
        df.columns = df.columns.map(transform_columns)

        # Remove values after the decimal point using a loop
        new_columns = []
        for col in df.columns:
            if '.' in str(col):
                new_col = str(col).split('.')[0]
                new_columns.append(new_col)
            else:
                new_columns.append(str(col))

        df.columns = new_columns

        #Find Index of ' ' column that will act as separator
        index = list(df.columns).index(' ')

        l = df.iloc[0, index+1:]         #Month names (first row elements)
        m = df.columns[index+1:]         #Years from column Header

        concatenated_columns_list = []
        for i in range(0,len(m)):
            concatenated_columns_list.append(l[i]+m[i])   #simple string concatenation

        unaltered_columns_list = list(df.columns[:index+1])    #Sender_name, Branch ID ....

        columns_list = unaltered_columns_list + concatenated_columns_list    #creating new final column list
        df.columns = columns_list

        # Drop the first row
        df = df.iloc[1:]

        # Drop the 5th column (index 5)
        df = df.drop(df.columns[4], axis=1)

        #df.pivot(index ='A', columns ='B', values =['C', 'A'])
        table = pd.pivot_table(df, values =df.columns[4:], index = 'BRANCH ID',aggfunc = np.sum)

        # Determine the current month as "Sep2023" (September 2023)
        today = datetime.now()-pd.offsets.DateOffset(months=1)
        currentdata_month = today.strftime('%b%Y')

        # Define the sequence of months to extract (13 months)
        sequence = [currentdata_month]
        for _ in range(1, 13):
            today -= relativedelta(months=1)
            sequence.append(today.strftime('%b%Y'))

        # Select the 13 months' data and reorder the columns
        selected_data = table[sequence]
        return selected_data
 # Call the function and store the result in a variable
result_selected_data = dq_non_trending_branch_analysis(branch_report_file_path, '', '')

# Now you can use 'result_selected_data' outside the function
print(result_selected_data)
    


      



Please enter correct path for Current Month Branch DQ file
Please enter correct path for Previous Month Branch DQ file
            Oct2023  Sep2023  Aug2023  Jul2023  Jun2023  May2023  Apr2023  \
BRANCH ID                                                                   
1205157153      727      666      829      651      729      656      570   

            Mar2023  Feb2023  Jan2023  Dec2022  Nov2022  Oct2022  
BRANCH ID                                                         
1205157153      563      479      541      451      403      420  


In [13]:


# Extract the first column of result_selected_data
result_selected_data_first_column = result_selected_data.index.tolist()

# Extract the first column of ins_pivot_branch
ins_pivot_branch_first_column = ins_pivot_branch.index.tolist()

# Check for common values
common_values = set(result_selected_data_first_column) & set(ins_pivot_branch_first_column)

# Add a new column in ins_pivot_branch with the comment "trending" or "not trending"
ins_pivot_branch['Trend_Status'] = 'Unreported'
ins_pivot_branch.loc[ins_pivot_branch.index.isin(common_values), 'Trend_Status'] = ''

# Print the modified ins_pivot_branch dataframe
print(ins_pivot_branch)



year            2022           2023                                          \
month             10   11   12    1    2    3    4    5    6    7    8    9   
(ID1_VALUE.1, )                                                               
FS2807088        186  182  194  226  177  211  254  291  240  246  314  208   
G24KKWMF1        231  254  251  372  254  328  321  395  575  276  505  353   

year                 Trend_Status  
month             10               
(ID1_VALUE.1, )                    
FS2807088        192   Unreported  
G24KKWMF1        510   Unreported  


In [19]:

# Total sum column
ins_pivot_branch['Total'] = 0
# Create a new column 'Comment' based on Total value
ins_pivot_branch['Comment'] = ''
ins_pivot_branch['Address'] = ''

ins_pivot_branch['Total'] = ins_pivot_branch.iloc[:, :-1].sum(axis=1)
ins_pivot_branch.loc[(ins_pivot_branch['Total'] < 100) & (ins_pivot_branch['Trend_Status'] == 'Unreported'), 'Comment'] = 'Pass as low volume'
ins_pivot_branch.loc[(ins_pivot_branch['Total'] > 100) & (ins_pivot_branch['Trend_Status'] == 'Unreported'), 'Comment'] = 'Over 100'
ins_pivot_branch


year            2022           2023                                          \
month             10   11   12    1    2    3    4    5    6    7    8    9   
(ID1_VALUE.1, )                                                               
FS2807088        186  182  194  226  177  211  254  291  240  246  314  208   
G24KKWMF1        231  254  251  372  254  328  321  395  575  276  505  353   

year                 Trend_Status Total   Comment Address  
month             10                                       
(ID1_VALUE.1, )                                            
FS2807088        192   Unreported  2921  Over 100          
G24KKWMF1        510   Unreported  4625  Over 100

In [15]:
l = list(ins_pivot_branch[ins_pivot_branch['Comment'] == 'Over 100'].index)
for branch in l:
    ins_pivot_branch.loc[branch,'Address'] = str(list(df_combined_ins[df_combined_ins['ID1_VALUE.1'] == branch]['ADDRESS1'].unique()))

ins_pivot_branch

year            2022           2023                                          \
month             10   11   12    1    2    3    4    5    6    7    8    9   
(ID1_VALUE.1, )                                                               
FS2807088        186  182  194  226  177  211  254  291  240  246  314  208   
G24KKWMF1        231  254  251  372  254  328  321  395  575  276  505  353   

year                 Trend_Status Total   Comment                    Address  
month             10                                                          
(ID1_VALUE.1, )                                                               
FS2807088        192   Unreported  2553  Over 100  ['15 EARHART DR STE 101']  
G24KKWMF1        510   Unreported  4140  Over 100  ['15 EARHART DR STE 101']

In [16]:


# Filter rows where the comment is 'Over 100'
over_100_data = ins_pivot_branch[ins_pivot_branch['Comment'] == 'Over 100']

# Extract values from the first column where the comment is 'Over 100'
values_over_100 = over_100_data.index.tolist()

# Displaying the values from the first column where the comment is 'Over 100'
print(values_over_100)



['FS2807088', 'G24KKWMF1']


In [17]:
values_over_100

['FS2807088', 'G24KKWMF1']

In [18]:


# Filter df_combined_ins based on ID1_VALUE matching values_over_100
filtered_addresses = df_combined_ins[df_combined_ins['ID1_VALUE.1'].isin(values_over_100)]

# Extract the 'ADDRESS1' values for the filtered rows
branch_addresses = filtered_addresses['ADDRESS1'].tolist()


# Display the extracted 'ADDRESS1' values
print(branch_addresses)



['15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101', '15 EARHART DR STE 101',